In [12]:
import math

def _square_residues_mod_2k(k, parity):
    if parity not in ('even', 'odd', 'any'):
        raise ValueError("parity must be 'even', 'odd' or 'any'")
    M = 1 << k
    res = set()
    if parity == 'any':
        for x in range(M):
            res.add((x * x) % M)
        return res
    start = 0 if parity == 'even' else 1
    for x in range(start, M, 2):
        res.add((x * x) % M)
    return res

def _sumset_mod(S, T, M):
    out = set()
    for a in S:
        aa = a
        for b in T:
            out.add((aa + b) % M)
    return out

def _build_suffix_sumsets_mod(parities, k):
    M = 1 << k
    S = [_square_residues_mod_2k(k, p) for p in parities]
    suffix_res = [set() for _ in range(5)]
    suffix_res[4] = {0}
    for i in range(3, -1, -1):
        suffix_res[i] = _sumset_mod(S[i], suffix_res[i+1], M)
    return suffix_res

def _has_solution_mod(n, parities, k):
    M = 1 << k
    r = n % M
    S = [_square_residues_mod_2k(k, p) for p in parities]
    A = _sumset_mod(S[0], S[1], M)
    B = _sumset_mod(S[2], S[3], M)
    for x in A:
        need = (r - x) & (M - 1)
        if need in B:
            return True
    return False

def _asc_min_sum_sq_from(index, prev_val, parities):
    total = 0
    seq = []
    cur = prev_val
    for j in range(index, 4):
        p = parities[j]
        cand = cur + 1
        if p == 'even':
            if cand % 2 == 1:
                cand += 1
        elif p == 'odd':
            if cand % 2 == 0:
                cand += 1
        total += cand * cand
        seq.append(cand)
        cur = cand
    return total, seq

def _desc_min_sum_sq_from(index, ub, parities):
    if index == 4:
        return 0, []
    vals = [None, None, None, None]  # значения на позициях index..3
    prev = -1
    for j in range(3, index - 1, -1):
        y = prev + 1
        p = parities[j]
        if p == 'even':
            if y % 2 == 1:
                y += 1
        elif p == 'odd':
            if y % 2 == 0:
                y += 1
        if y <= prev:
            y = prev + 1
            if p == 'even' and y % 2 == 1:
                y += 1
            if p == 'odd' and y % 2 == 0:
                y += 1
        vals[j] = y
        prev = y

    first = vals[index]
    if ub != float('inf') and first >= ub:
        return float('inf'), None

    total = 0
    seq = []
    for j in range(index, 4):
        total += vals[j] * vals[j]
        seq.append(vals[j])
    return total, seq

def _find_ordered_four_squares_core_desc(n, parities, k=7):
    """
    Поиск по убыванию: a1 > a2 > a3 > a4.
    parities[i] ∈ {'even','odd','any'}.
    """
    if n < 0:
        return None
    for p in parities:
        if p not in ('even', 'odd', 'any'):
            raise ValueError("each parity must be 'even','odd' or 'any'")

    if not _has_solution_mod(n, parities, k):
        return None

    M = 1 << k
    suffix_res = _build_suffix_sumsets_mod(parities, k)

    tail0, _ = _desc_min_sum_sq_from(0, float('inf'), parities)
    if tail0 > n:
        return None

    solution = None
    max_root = int(math.isqrt(n))

    def dfs(i, prev_ub, cur_sum, chosen):
        nonlocal solution
        if solution is not None:
            return
        if i == 4:
            if cur_sum == n:
                solution = tuple(chosen)
            return

        tail_min, _ = _desc_min_sum_sq_from(i, prev_ub, parities)
        if tail_min == float('inf') or cur_sum + tail_min > n:
            return

        r_need_total = (n - cur_sum) & (M - 1)
        if r_need_total not in suffix_res[i]:
            return

        p = parities[i]
        start = max_root if prev_ub == float('inf') else min(max_root, prev_ub - 1)
        if start < 0:
            return
        x = start
        if p == 'even':
            if x % 2 == 1:
                x -= 1
            step = -2
        elif p == 'odd':
            if x % 2 == 0:
                x -= 1
            step = -2
        else:
            step = -1

        while x >= 0:
            s = cur_sum + x*x

            rest_min, _ = _desc_min_sum_sq_from(i+1, x, parities)
            if s + rest_min > n:
                x += step
                continue

            r_need = (n - s) & (M - 1)
            if r_need in suffix_res[i+1]:
                chosen.append(x)
                dfs(i+1, x, s, chosen)
                chosen.pop()
                if solution is not None:
                    return

            x += step

    dfs(0, float('inf'), 0, [])
    return solution

def find_ordered_four_squares_with_lagrange(n, p1, p2, p3, p4, k=7):
    """
    Ищет n = a1^2 + a2^2 + a3^2 + a4^2
    при чётностях p_i ∈ {'even','odd'} и строгом порядке a1 > a2 > a3 > a4 ≥ 0.
    Включает редукцию Лагранжа (деление на 4^t), когда ВСЕ p_i = 'even'.
    """

    if n < 0:
        return None

    parities_req = [p1, p2, p3, p4]
    for p in parities_req:
        if p not in ('even', 'odd'):
            raise ValueError("p_i must be 'even' or 'odd'")

    all_even = all(p == 'even' for p in parities_req)

    # Ветвь с редукцией Лагранжа (только если все четные и n кратно 4)
    if all_even:
        # Необходимое условие: n ≡ 0 (mod 4)
        if n % 4 != 0:
            return None

        # Делим на максимальную степень 4
        t = 0
        m = n
        while m % 4 == 0:
            m //= 4
            t += 1

        par_any = ['any', 'any', 'any', 'any']
        sol_b = _find_ordered_four_squares_core_desc(m, par_any, k=k)
        if sol_b is None:
            return None

        scale = 1 << t
        a1, a2, a3, a4 = (sol_b[0]*scale, sol_b[1]*scale, sol_b[2]*scale, sol_b[3]*scale)

        # Финальная проверка чётности
        if (a1 % 2 != 0) or (a2 % 2 != 0) or (a3 % 2 != 0) or (a4 % 2 != 0):
            return None

        # Проверка строгого убывания
        if not (a1 > a2 > a3 > a4):
            return None

        return (a1, a2, a3, a4)

    # Обычная ветвь без редукции
    return _find_ordered_four_squares_core_desc(n, parities_req, k=k)

In [17]:
def print_quaternion_report(n, parities, result):
    print("=" * 79)
    print("АЛГОРИТМ ПОИСКА КВАТЕРНИОННОГО РАЗЛОЖЕНИЯ")
    print("С ЗАДАННЫМИ ОГРАНИЧЕНИЯМИ ЧЁТНОСТИ")
    print("В КАНОНИЧЕСКОЙ ФОРМЕ: a >= b >= c >= d >= 0")
    print("=" * 79)
    print()
    print()
    print("=" * 70)

    if result is None:
        print(f"n={n}, parities={parities}")
        print("Результат: Разложение не найдено.")
        return

    a, b, c, d = result
    print(f"Check=({a}, {b}, {c}, {d})")
    print(f"n={n}, parities={parities}")
    print(f"Результат: Число: n = {n}")
    print(f"Ограничения чётности: {parities}")
    print(f"n mod 8 = {n % 8}")
    k = sum(1 for p in parities if p == 'odd')
    print(f"Количество нечётных коэффициентов: k = {k}")
    exists = True
    print(f"Существование: {'✓ ДА' if exists else '✗ НЕТ'}")
    print()
    print(f"Каноническая форма: ({a}, {b}, {c}, {d})")
    print(f"  {n} = {a}² + {b}² + {c}² + {d}²")
    print(f"  {n} = {a**2} + {b**2} + {c**2} + {d**2}")
    print(f"  Проверка упорядочения: {a} >= {b} >= {c} >= {d}? {a >= b >= c >= d}")
    print()
    actual_parities = ['odd' if x % 2 else 'even' for x in (a, b, c, d)]
    print(f"Фактическая чётность: {actual_parities}")
    print(f"Соответствие: {'✓' if actual_parities == parities else '✗'}")
    print()

# Пример использования:

n1 = 7321
p1 = ['odd', 'even', 'even', 'even']
result = find_ordered_four_squares_with_lagrange(n1, *p1, k=7)
print_quaternion_report(n1,p1, result)

АЛГОРИТМ ПОИСКА КВАТЕРНИОННОГО РАЗЛОЖЕНИЯ
С ЗАДАННЫМИ ОГРАНИЧЕНИЯМИ ЧЁТНОСТИ
В КАНОНИЧЕСКОЙ ФОРМЕ: a >= b >= c >= d >= 0


Check=(81, 20, 18, 6)
n=7321, parities=['odd', 'even', 'even', 'even']
Результат: Число: n = 7321
Ограничения чётности: ['odd', 'even', 'even', 'even']
n mod 8 = 1
Количество нечётных коэффициентов: k = 1
Существование: ✓ ДА

Каноническая форма: (81, 20, 18, 6)
  7321 = 81² + 20² + 18² + 6²
  7321 = 6561 + 400 + 324 + 36
  Проверка упорядочения: 81 >= 20 >= 18 >= 6? True

Фактическая чётность: ['odd', 'even', 'even', 'even']
Соответствие: ✓



In [23]:
n1 = 30
p1 = ['odd','even','odd','even']
result = find_ordered_four_squares_with_lagrange(n1, *p1, k=7)
print_quaternion_report(n1,p1, result)

АЛГОРИТМ ПОИСКА КВАТЕРНИОННОГО РАЗЛОЖЕНИЯ
С ЗАДАННЫМИ ОГРАНИЧЕНИЯМИ ЧЁТНОСТИ
В КАНОНИЧЕСКОЙ ФОРМЕ: a >= b >= c >= d >= 0


Check=(5, 2, 1, 0)
n=30, parities=['odd', 'even', 'odd', 'even']
Результат: Число: n = 30
Ограничения чётности: ['odd', 'even', 'odd', 'even']
n mod 8 = 6
Количество нечётных коэффициентов: k = 2
Существование: ✓ ДА

Каноническая форма: (5, 2, 1, 0)
  30 = 5² + 2² + 1² + 0²
  30 = 25 + 4 + 1 + 0
  Проверка упорядочения: 5 >= 2 >= 1 >= 0? True

Фактическая чётность: ['odd', 'even', 'odd', 'even']
Соответствие: ✓

